In [34]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobClient

account_url = f"https://wboylesbackups.blob.core.windows.net"
credential = DefaultAzureCredential()

def download_blob(container_name: str, blob_name: str, out_filename: str | None = None):
    if out_filename is None:
        out_filename = container_name + blob_name

    blob_client = BlobClient(account_url, container_name, blob_name, credential=credential)

    with open(out_filename, "wb") as f:
        f.write(blob_client.download_blob().readall())


In [33]:
container_name = "videos"
blob_name = "20220806_132412.jpg"
download_blob(container_name, blob_name)

In [69]:
from azure.storage.blob import BlobPrefix, ContainerClient, BlobProperties

# Returns list of BlobProperties in preorder traversal
# Does not include folders, only files
def list_blobs(container_name: str, prefix: str="") -> list[BlobProperties]:
    blobs: list[BlobProperties] = []
    container_client = ContainerClient(account_url, container_name, credential)

    for blob in container_client.walk_blobs(name_starts_with=prefix):
        name: str = blob.name # type: ignore

        if isinstance(blob, BlobPrefix):
            blobs += list_blobs(container_name, prefix=name)
        else:
            blobs.append(blob)


    return blobs

[{'name': 'folder1/20220806_132412.jpg', 'container': 'videos', 'snapshot': None, 'version_id': None, 'is_current_version': None, 'blob_type': <BlobType.BLOCKBLOB: 'BlockBlob'>, 'metadata': {}, 'encrypted_metadata': None, 'last_modified': datetime.datetime(2023, 4, 17, 0, 51, 29, tzinfo=datetime.timezone.utc), 'etag': '0x8DB3EDDE184A792', 'size': 5652690, 'content_range': None, 'append_blob_committed_block_count': None, 'is_append_blob_sealed': None, 'page_blob_sequence_number': None, 'server_encrypted': True, 'copy': {'id': None, 'source': None, 'status': None, 'progress': None, 'completion_time': None, 'status_description': None, 'incremental_copy': None, 'destination_snapshot': None}, 'content_settings': {'content_type': 'image/jpeg', 'content_encoding': None, 'content_language': None, 'content_md5': bytearray(b':\xf1\nL\xd2\xeeFx\x97\x99\x94;D\xb9g^'), 'content_disposition': None, 'cache_control': None}, 'lease': {'status': 'unlocked', 'state': 'available', 'duration': None}, 'blob